# Comment by HHS:
# Only small changes were made to reyers code to specify the correct paths for my project.
# It was also moved over to a jupyter notebook because of my personal preference.

In [3]:
import wave, struct, os, math
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
def load_samples(path: str) -> any:
    with wave.open(path, 'rb') as audio:
        frames = audio.getnframes()
        frame_bytes = audio.readframes(frames)
        samples = [ struct.unpack("<h", frame_bytes[(i):(2+i)]) for i in range(0, frames, 2) ] 
        return np.array(samples)


In [5]:
def compute_mse(original: np.ndarray, changed: np.ndarray):
    assert len(original) == len(changed)
    n = len(original)
    data = (original - changed) ** 2
    return (sum(data) / n)[0]

In [6]:
def compute_snr(original: np.ndarray, mse: int):
    n = len(original)
    data = original ** 2
    signal = (sum(data)[0] / n)
    return 10 * math.log10(signal / mse)

In [7]:
def compute_method_transparency(base_url, method_name, original_url):
    data = list()

    for file in tqdm(os.listdir(original_url), f'[*] Computing perceptual transparency for {method_name}'):
        original_samples = load_samples(original_url+file)
        stego_samples = load_samples(base_url+'/'+file)

        genre = file.split('.')[0]

        mse = compute_mse(original_samples, stego_samples)
        snr = compute_snr(original_samples, mse)

        data.append([file, genre, mse, snr])
    
    df = pd.DataFrame(data, columns=['File', 'Genre', 'MSE', 'SNR'])
    return df

In [24]:
def compute_transparency():
    # Steghide
    #df = compute_method_transparency('audio/steghide16hz', 'steghide', original_url='audio/steg4pgpref16hz/')
    #df.to_csv('results/perceptual_transparency_steghide.csv')

    # Hide4PGP
    #df = compute_method_transparency('audio/hide4pgp16hz', 'hide4pgp', original_url='audio/steg4pgpref16hz/')
    #df.to_csv('results/perceptual_transparency_hide4pgp.csv')

    # GAN High
    #df = compute_method_transparency('audio/ganhigh44hz', 'ganhigh', original_url='audio/ganref44hz/')
    #df.to_csv('results/perceptual_transparency_ganhigh.csv')

    # Gan Low
    df = compute_method_transparency('audio/ganlow44hz', 'ganlow', original_url='audio/ganref44hz/')
    df.to_csv('results/perceptual_transparency_ganlow.csv')

In [25]:
compute_transparency()

[*] Computing perceptual transparency for ganlow: 100%|██████████| 8/8 [00:02<00:00,  3.42it/s]
